In [16]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [17]:
import numpy as np
from base.MachiKoro.env import *
#0 Array bias trong mỗi trận
#1 Array bias sau 100k trận 
#2 Lưu số trận 
#3 lưu hệ số đã dùng 
#4 Tỷ lệ thắng khi đấu 10k trận 
#5 Trigger để train 100k trận 
#6 Đếm số lần yếu liên tiếp
#7 Lưu array cho policy yếu 
#8 State của trận đấu
#9 Action của trận đấu
#10 Reward của trận đấu
#11 Số trận lưu state
#12 Trigger policy yếu

perx = [np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,getActionSize())),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,2)),np.zeros((1,2)),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((2000000,getStateSize())),np.zeros((2000000,1)),np.zeros((2000000,1)),np.zeros((1,1)),np.zeros((1,2))]
                         #0                                                      #1                   #2              #3                                                            #4              #5               #6                #7                                                               #8                                   #9               #10                       #11           #12

# Các hàm check tính chất của state


In [18]:
import numpy as np
@njit
def check_min(state, reward):
    just = np.zeros((2000000,2))
    for i in range(len(reward)):
        if reward[i] == -1:
            for j in range(i,len(reward)):
                if reward[j] != -1:
                    reward[i] = reward[j]
                    break
    # replace -1 with 0
    for i in range(len(reward)):
        if reward[i] == -1:
            reward[i] = 0
    # np.mean each array in the state and the corresponding reward
    for i in range(len(reward)):
        just[i][0] = np.mean(state[i])
        just[i][1] = (reward[i][0])
    # count the number of times each value of the state appears and the reward of each value of the state
    count = np.zeros((2000000,3))
    for i in range(len(reward)):
        for j in range(len(count)):
            if just[i][0] == count[j][0]:
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
            if count[j][0] == 0:
                count[j][0] = just[i][0]
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
    # find the index value the lowest rate of reward has the highest number of times appear
    min = 0
    for i in range(len(count)):
        if count[i][1] > count[min][1]:
            min = i
    for i in range(len(count)):
        if count[i][1] == count[min][1] and count[i][2] < count[min][2]:
            min = i

    return count[min][0]
@njit
def check_weaker(min,x):
    if x == min:
        return 1
    else:
        return 0


# Agent Hoàn Chỉnh

In [19]:
@njit()
def p_bias(state,per):
    # Chuyển Mode gồm 3 mode mode 0 - train bias , mode 1 - test bias , mode 2 - train policy yếu ,mode 3 - test policy yếu,mode 4 - lưu state
    if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      # Lưu lại kết quả thắng
      per[5][0][1] = 0 
      per[5][0][1] += (per[4][0][0])
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Reset số lần yếu liên tiếp
      per[6][0][0] = 0 
    elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[6][0][0] +=1 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 

    # Bắt đầu mode lưu state
    if per[6][0][0] == 3 and per[11][0][0] < 2000000 :
      # Chuyển sang lưu state 
      per[5][0][0] = 4
      per[2][0][0] = 0
      # Rest số trận thắng và số trận đấu 
      per[4][0][1] = 0 
      per[4][0][0] = 0
      
    # Hết mode lưu state chuyển sang train policy yếu
    if per[5][0][0] == 4 and per[11][0][0] == 2000000:
      per[5][0][0] = 2

    # Bất đầu mode train và test policy yếu hơn
    if per[11][0][0] == 2000000:
      if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
        # Lưu lại kết quả thắng
        per[5][0][1] = 0 
        per[5][0][1] += (per[4][0][0])
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))

      elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
      
    # reset per[1][0] sau 100k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] < 2000000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 1
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] == 2000000:
      per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 3
      
    if per[11][0][0] == 2000000 and per[12][0][0] == 0:
      per[12][0][0] += check_min(per[8], per[10])

    # Chế độ train bias
    if int(per[2][0][0]) < 100000 and per[5][0][0] == 0  :
      actions = getValidActions(state)
      actions *= per[0][0]
      action = np.argmax(actions)
      if getReward(state) == 1 :
          per[1] += per[0][0]
      if getReward(state) == 0:
          per[0][0] = np.random.rand(getActionSize())

    # Chế độ test bias
    elif int(per[2][0][0]) <10000  and per[5][0][0] == 1 : 
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0])    
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  

    # Chế độ train policy yếu
    elif int(per[2][0][0]) < 100000 and per[5][0][0] == 2 and per[11][0][0] == 2000000 :
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      Check_yeu = check_weaker(per[12][0][0],np.mean(state))
      if Check_yeu == 1:
        per[12][0][1] = Check_yeu
        actions *= per[0][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1 and per[12][0][1] == 1:
        per[1] += per[0][0]
        per[12][0][1] = 0 
      if getReward(state) == 0 and per[12][0][1] == 1:
        per[0][0] = np.random.rand(getActionSize())
        per[12][0][1] = 0 

    # chế độ test policy yếu
    elif int(per[2][0][0]) <10000 and  per[5][0][0] == 3 and per[11][0][0] == 2000000:
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      if check_weaker(per[12][0][0],np.mean(state)) == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  
    

    # Báo chuyển sang chế độ lưu state và action
    if per[5][0][0] == 4 and per[11][0][0] < 2000000:
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0]) 


    # Lưu state và action
    if getReward(state) != -2 and per[11][0][0] < 2000000 and per[5][0][0] == 4:
        for i in range(getStateSize()):
          per[8][int(per[11][0][0])][i] += state[i]        
        per[9][int(per[11][0][0])][0] += int(action)
        per[10][int(per[11][0][0])][0] += int(getReward(state))
        per[11][0][0] += 1

    # đếm số ván đấu 
    if getReward(state) != -1 :
        per[2][0][0] += 1
    return action,per

win, bias = numba_main_2(p_bias, 1000000,perx,0)

In [ ]:
perx[3]

array([[0.79784021, 0.75199562, 0.82261278, 0.78838267, 0.79052828,
        0.79565535, 0.79837103, 0.79578274, 0.79445001, 0.7821219 ,
        0.77975026, 0.78846079, 0.78619977, 0.7929032 , 0.79804748,
        0.78130051, 0.79366103, 0.7936773 , 0.7845971 , 0.79723485,
        0.79205496, 0.79062392, 0.79292246, 0.80331379, 0.79024906,
        0.78935863, 0.77357922, 0.79596245, 0.78999221, 0.79677858,
        0.7908595 , 0.79397239, 0.77996034, 0.79500048, 0.93091332,
        1.        , 0.78042079, 0.78912611, 0.85421328, 0.89404385,
        0.66803933, 0.72304435, 0.78471488, 0.71294605, 0.76366293,
        0.72248261, 0.80064936, 0.78674514, 0.73705281, 0.92741798,
        0.91809799, 0.95155101, 0.87150687, 0.25124413]])

In [ ]:
perx[7]


array([[0.96991   , 0.99248568, 0.95486049, 0.99154653, 0.98077082,
        0.99291677, 0.98655239, 0.98045413, 0.9713485 , 0.96958679,
        0.98224289, 0.98426915, 0.98311073, 1.        , 0.97282256,
        0.96259567, 0.97581814, 0.97845296, 0.98880782, 0.97661965,
        0.98473322, 0.98093194, 0.97338918, 0.98577039, 0.98692466,
        0.98809835, 0.99660929, 0.98383068, 0.97965674, 0.98879893,
        0.98717227, 0.97375038, 0.98614717, 0.97603493, 0.99377993,
        0.98944801, 0.97965767, 0.99672918, 0.99627906, 0.98726773,
        0.96888597, 0.99473151, 0.9938202 , 0.96784152, 0.99828557,
        0.99221192, 0.9804512 , 0.9638414 , 0.97689938, 0.98558521,
        0.98639662, 0.98891658, 0.96851047, 0.96823312]])

In [ ]:
perx[12]

array([[0.94444444, 0.        ]])

# Test lại tỷ lệ thắng của level 1 game MachiKoro

**Khi chưa khắc phục policy yếu :24551** 


**Khi đã khắc phục policy yếu :27961**

In [ ]:
@njit()
def test(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    if check_weaker(per[12][0][0],np.mean(state)) == 1:
      actions *= per[7][0]
      action = np.argmax(actions)
    else:
      actions *= per[3][0]
      action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(test, 1000,perx,1)
print(win)

0
